In [1]:
from typing import List

import numpy as np

from nn.nn import Value
from nn.models import Perceptron

In [2]:
def make_perceptron_value(data: List[Value], weights: List[Value], bias: Value) -> Value:
    """Create and return a perceptron

    Parameters
    ----------
    data: list of Value objects (len should be >= 1)
        each entry represents a feature of x
    weights: list of Value objects (len should be >= 1)
        list of weights
    bias: Value
        represents a bias term
    """
    summands = map(lambda d, w: d * w, data, weights)
    
    linear_comb = next(summands)
    for summand in summands:
        linear_comb = linear_comb + summand

    logit = linear_comb + bias
    
    probability = logit.sigmoid()

    return probability

def make_binary_crossentropy_loss(prediction_values: List[Value], ground_truth: List[float]):
    loss = 0
    for p, g in zip(prediction_values, ground_truth):
        if np.isclose(g, 0):
            example_cost = (1 - p).log()
        elif np.isclose(g, 1):
            example_cost = p.log()
        loss = loss + example_cost
    loss = (-1) * loss
    return loss
#TODO: write log

This is an example that creates loss function:

In [3]:
weights = [Value(0), Value(1)]
bias = Value(2)

data1 = [Value(3), Value(4)]
perceptron1 = make_perceptron_value(data1, weights, bias)

data2 = [Value(0), Value(0)]
perceptron2 = make_perceptron_value(data2, weights, bias)

loss = make_binary_crossentropy_loss([perceptron1, perceptron2], [1, 0])

This example demonstrates how to train a perceptron to behave like an AND gate:

In [4]:
X = [
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1],
    ]

Y = [0, 0, 0, 1]

perceptron = Perceptron(no_weights=2)

def simple_gradient_descent_iteration(model, X, Y, alpha=1e-2):
    for parameter in model.parameters():
        parameter.grad = 0

    loss = sum((model(x) - y)**2 for x, y in zip(X, Y))

    Value.back_prop(loss)
    print("Loss =", loss.data)

    for parameter in model.parameters():
        parameter.data -= alpha * parameter.grad

num_iterations = 100

for i in range(num_iterations):
    print(f"Iteration {i}")
    simple_gradient_descent_iteration(perceptron, X, Y, 1)
    print("y_pred =", *[perceptron(x).data for x in X])

Iteration 0
Loss = 0.8523001622353087
y_pred = 0.2975929795474892 0.215075028131512 0.40254539200658424 0.3035001821013969
Iteration 1
Loss = 0.7819736381602448
y_pred = 0.27788458503673213 0.23704522675989487 0.4036627758292637 0.35338448797326316
Iteration 2
Loss = 0.7144655391144722
y_pred = 0.25901885709056516 0.25821321277798914 0.40488487928699657 0.40387282607701275
Iteration 3
Loss = 0.6530642045462869
y_pred = 0.23912437719267698 0.27417204218395824 0.4014003348186365 0.4462819000905386
Iteration 4
Loss = 0.6000767394429656
y_pred = 0.21876908859016816 0.28406564084853114 0.3931158181405811 0.47857241765076836
Iteration 5
Loss = 0.5549799725401626
y_pred = 0.19932534308570896 0.28959482274208664 0.3824323426069891 0.5034816296850491
Iteration 6
Loss = 0.5163807424873403
y_pred = 0.1815780707775113 0.29246584141306775 0.3712612555775008 0.5238430185171598
Iteration 7
Loss = 0.48306725508856885
y_pred = 0.1656899416314183 0.29364732849156266 0.3604546020294887 0.5412460898996734

/home/bohdan/MFML/NN/nn/nn.py:77: RuntimeWarning: invalid value encountered in log
  local_grad_wrt_exponent = np.log(self.data) * out.data
